# Create dissolved Pb forcing in runoff
First create a river classification system based on runoff type and then apply concentrations

In [2]:
import numpy as np
import xarray as xr
import sys
sys.path.append('../paper-materials/')
from constants import imin, imax, jmin, jmax

%matplotlib inline

#### Load files

In [3]:
# Load the runoff forcing used in ANHA12 to identify the runoff locations
# based on Dai, Trenberth et al. (2009), and Bamber et al. (2012)
c = xr.open_dataset('/ocean/brogalla/GEOTRACES/data/runoff/ANHA12_runoff_monthly_combined_Dai_Trenberth_Bamber_y2015.nc')
lon_rf  = c['nav_lon'].values
lat_rf  = c['nav_lat'].values
socoefr = c['socoefr'].values
rf      = c['runoff'][8].values

# mask places where runoff is zero
rf[rf == 0]       = np.nan
lon_rf[rf == 0.0] = np.nan
lat_rf[rf == 0.0] = np.nan

Classify runoff based on:
1. Glaciers
2. Continental
3. other

Start off with classifying everything as other:

In [9]:
class_river = np.empty(rf.shape)
class_river[:] = np.nan
class_river[~np.isnan(rf)] = 3.0

then identify glacier runoff: 

In [10]:
# array of indices where latitude  & longitude condition (Greenland) is met
index1 = np.argwhere((lat_rf < 82) & (lat_rf > 73) & (lon_rf < -62) & (lon_rf > -74))
index2 = np.argwhere((lat_rf < 85) & (lat_rf > 76) & (lon_rf < -75) & (lon_rf > -85))
index3 = np.argwhere((lat_rf < 74) & (lat_rf > 71) & (lon_rf < -70) & (lon_rf > -83))
index4 = np.argwhere((lat_rf < 77) & (lat_rf > 73) & (lon_rf < -79) & (lon_rf > -84))
index5 = np.argwhere((lat_rf < 80) & (lat_rf > 78) & (lon_rf < -90) & (lon_rf > -93))
index6 = np.argwhere((lat_rf < 84) & (lat_rf > 79) & (lon_rf < -93) & (lon_rf > -97))
index7 = np.argwhere((lat_rf < 86) & (lat_rf > 73) & (lon_rf < -40) & (lon_rf > -62))

index8 = np.argwhere((lat_rf < 86) & (lat_rf > 55) & (lon_rf < 20) & (lon_rf > -62))
index9 = np.argwhere((lat_rf < 75) & (lat_rf > 63) & (lon_rf < -50) & (lon_rf > -71))

In [11]:
def classify_glacier(index, rf, class_river):
    for i in range(0,index.shape[0]):         
        if ~np.isnan(rf[index[i][0],index[i][1]]):
#             rf[index[i][0],index[i][1]] = 1.0
            class_river[index[i][0],index[i][1]] = 1.0
    return class_river

In [12]:
class_river = classify_glacier(index1, rf, class_river) #Greenland
class_river = classify_glacier(index2, rf, class_river) #Ellesmere Island
class_river = classify_glacier(index3, rf, class_river) #Baffin Island
class_river = classify_glacier(index4, rf, class_river) #Southern Ellesmere Island
class_river = classify_glacier(index5, rf, class_river) #Western Ellesmere Island
class_river = classify_glacier(index6, rf, class_river) #Western Ellesmere Island
class_river = classify_glacier(index7, rf, class_river) #More of Greenland
class_river = classify_glacier(index8, rf, class_river) #S Greenland
class_river = classify_glacier(index9, rf, class_river) #S Baffin Island

and then identify continental drainage:

In [13]:
def classify_continental(index, rf, class_river):
    for i in range(0,index.shape[0]):         
        if ~np.isnan(rf[index[i][0],index[i][1]]):
#             rf[index[i][0],index[i][1]] = 2.0
            class_river[index[i][0],index[i][1]] = 2.0
    return class_river

In [14]:
# array of indices where latitude  & longitude condition (Greenland) is met
index1 = np.argwhere((lat_rf < 71) & (lat_rf > 67) & (lon_rf < -118) & (lon_rf > -150))
index2 = np.argwhere((lat_rf < 68) & (lat_rf > 67) & (lon_rf < -95) & (lon_rf > -115))
index3 = np.argwhere((lat_rf < 70) & (lat_rf > 67) & (lon_rf < -88) & (lon_rf > -100))
index4 = np.argwhere((lat_rf < 70) & (lat_rf > 65) & (lon_rf < -81) & (lon_rf > -87))
index5 = np.argwhere((lat_rf < 69) & (lat_rf > 65) & (lon_rf < -80) & (lon_rf > -95))
index6 = np.argwhere((lat_rf < 69) & (lat_rf > 67) & (lon_rf < -104) & (lon_rf > -109))
index7 = np.argwhere((lat_rf < 69) & (lat_rf > 68) & (lon_rf < -116) & (lon_rf > -119))
index8 = np.argwhere((lat_rf < 69.2) & (lat_rf > 68) & (lon_rf < -113) & (lon_rf > -116))

index9 = np.argwhere((lat_rf < 62) & (lon_rf > -100) & (lon_rf < -50))
index10 = np.argwhere((lat_rf < 68) & (lat_rf > 60) & (lon_rf > -130) & (lon_rf < -88))
index11 = np.argwhere((lat_rf < 65) & (lat_rf > 64) & (lon_rf > -88) & (lon_rf < -87))

In [15]:
class_river = classify_continental(index1, rf, class_river) 
class_river = classify_continental(index2, rf, class_river)
class_river = classify_continental(index3, rf, class_river)
class_river = classify_continental(index4, rf, class_river)
class_river = classify_continental(index5, rf, class_river)
class_river = classify_continental(index6, rf, class_river)
class_river = classify_continental(index7, rf, class_river)
class_river = classify_continental(index8, rf, class_river)
class_river = classify_continental(index9, rf, class_river)
class_river = classify_continental(index10, rf, class_river)
class_river = classify_continental(index11, rf, class_river)

In [16]:
def classify_gulf(index, rf, class_river):
    for i in range(0,index.shape[0]):         
        if ~np.isnan(rf[index[i][0],index[i][1]]):
            class_river[index[i][0],index[i][1]] = 4.0
    return class_river

In [17]:
index1 = np.argwhere((lat_rf < 70) & (lat_rf > 69) & (lon_rf > -97) & (lon_rf < -94))

In [18]:
class_river = classify_gulf(index1, rf, class_river)

Consistency check:

In [19]:
print(rf[~np.isnan(rf)].shape, class_river[~np.isnan(class_river)].shape)

(18818,) (18818,)


#### Write classification to file:

In [17]:
file_write = xr.Dataset(
    {'rclass': (("y","x"), class_river)}, 
    coords = {
        "y": np.zeros(2400),
        "x": np.zeros(1632),
    })
    
file_write.to_netcdf('/ocean/brogalla/GEOTRACES/data/Pb-202207-forcing/river_class-202207.nc')

### Apply concentrations to classification

In [20]:
# Observations of dissolved Pb concentrations and suspended particulate material in small rivers in Inuit Nunangat
# from Colombo et al. (2019) and Brown et al. (2020)
river_names = ['Akpat Kuunga', 'Glacier', 'Marcil Creek', 'Saaqu', 'Devon Isl.', \
               'Cunningham', 'Garnier', 'Mecham', 'Creswell', 'Le Feuvre Inlet', \
               'Pasley', 'Simpson', 'Ellice', 'Tree']
river_lats = [73.7638, 73.6124, 72.9971, 73.7936, 74.5110, \
              74.0153, 73.9445, 74.6926, 72.8491, 72.3412, \
              70.4696, 67.6763, 67.8930, 67.6242]
river_lons = [-80.4468, -78.6805, -84.9864, -86.9829, -88.4910, -93.6242, -92.0369, -94.7783, -93.4162, \
             -96.9172, -95.5796, -100.5683, -104.0758, -111.9161]
river_spm = [8.31, 261, 3.18, np.nan, np.nan, \
             0.85, 13.6, 0.14, 2.43, np.nan, \
             0.03, 19.1, 4.82, np.nan]
river_dpb = [(1.53+3.20)/2, 35, (3.23+5.22)/2, (1.01+1.26)/2, 1.74, (0.601+0.670)/2, \
            3.53, 1.49, (4.48+6.07)/2, (1.59+1.78)/2, 1.72, (32+73)/2, \
            (60+64)/2, (12+15)/2]

Groupings:
- Glacial rivers: Glacial River
- Continental rivers: Tree River, Ellice River, Simpson River
- Other rivers: Marcil, Saaqu, Devon Island, Cunningham, Garnier, Mecham, Creswell, Le Feuvre Inlet, Pasle, Akput Kuunga

In [21]:
spm_glaciers = river_spm[1]
spm_other    = np.append(river_spm[2:11], river_spm[0])
spm_cont     = river_spm[11:14]

dPb_glaciers = river_dpb[1]
dPb_other    = np.append(river_dpb[2:11], river_dpb[0])
dPb_cont     = river_dpb[11:14]

In [22]:
print(f'dPb, glaciers: {np.average(dPb_glaciers):.2f}')
print(f'dPb, continental: {np.average(dPb_cont):.2f}')
print(f'dPb, other: {np.average(dPb_other):.2f}')
print('---------------------------------')
print(f'spm, glaciers: {np.nanmean(spm_glaciers):.2f}')
print(f'spm, continental: {np.nanmean(spm_cont):.2f}')
print(f'spm, other: {np.nanmean(spm_other):.2f}')

dPb, glaciers: 35.00
dPb, continental: 42.67
dPb, other: 2.38
---------------------------------
spm, glaciers: 261.00
spm, continental: 11.96
spm, other: 4.08


In [ ]:
# write these concentrations to the forcing file:
file_write = xr.Dataset(
    {'nav_lon': (("y","x"), lon_rf), 
     'nav_lat': (("y","x"), lat_rf),
     'spm_rivers': (("y","x"), SPM_rivers),
     'dpb_rivers': (("y","x"), dPb_rivers)}, 
     coords = {
         "y": np.zeros(2400),
         "x": np.zeros(1632),
    })
    
file_write.to_netcdf('/ocean/brogalla/GEOTRACES/data/Pb-tuning-202302/river-forcing-Pb-base-20230213.nc')